In [39]:
from models.FF_1H import *
import torch
from main import *
from torch.utils.data import DataLoader
from utils.Standard_Norm import *
from sklearn.preprocessing import StandardScaler
torch.manual_seed(0)

In [8]:
import numpy as np
import pandas as pd

def process_csv_file(file_path):
    # Load the CSV file using pandas
    data_frame = pd.read_csv(file_path, header=None)

    # Extract the values and reshape into a numpy array
    array_data = data_frame.values.reshape(-1)[:1024]

    # Create the final numpy array of length 1024, padded with zeros if necessary
    final_array = np.zeros(1024)
    final_array[:array_data.shape[0]] = array_data

    return final_array

In [12]:
shape0 = np.hstack((process_csv_file('check_shapes/shape 0/Given Yant0.csv'), process_csv_file('check_shapes/shape 0/Given Zant0.csv')))
shape0_opt = np.hstack((process_csv_file('check_shapes/shape 0/Optimized Yant0.csv'), process_csv_file('check_shapes/shape 0/Optimized Zant0.csv')))
shape1 = np.hstack((process_csv_file('check_shapes/shape 1/Given Yant1.csv'), process_csv_file('check_shapes/shape 1/Given Zant1.csv')))
shape1_opt = np.hstack((process_csv_file('check_shapes/shape 1/Optimized Yant1.csv'), process_csv_file('check_shapes/shape 1/Optimized Zant1.csv')))
shape2 = np.hstack((process_csv_file('check_shapes/shape 2/Given Yant2.csv'), process_csv_file('check_shapes/shape 2/Given Zant2.csv')))
shape2_opt = np.hstack((process_csv_file('check_shapes/shape 2/Optimized Yant2.csv'), process_csv_file('check_shapes/shape 2/Optimized Zant2.csv')))


In [19]:
import torch
import numpy as np
import pickle

def pred_shape(model, inp, scale):
    inp = torch.from_numpy(inp).float()
    out = model(inp).detach().numpy()
    return scale.inverse_transform(out.reshape(-1,1)).flatten()


In [20]:
def calculate_min_distance(ants, unit = 1):
    """
    Checks the smallest Euclidean distance, if it violates then return True
    """
    reshaped_tensor = torch.reshape(ants, (2, 1024))

# Transpose the reshaped tensor
    reshaped_tensor = torch.transpose(reshaped_tensor, 0, 1)
    nonzero_rows = torch.any(reshaped_tensor != 0, dim=1)
    filtered_tensor = reshaped_tensor[nonzero_rows]
    distances = torch.cdist(filtered_tensor, filtered_tensor)
    distances.fill_diagonal_(float('inf'))
    closest_neighbor_distances, _ = torch.min(distances, dim=1)
    smallest = torch.min(closest_neighbor_distances)
    return smallest.item()
    

In [32]:
def gradient_wrt_input(model, ants, lr, noise_std, iters):
    model.eval()
    last = None
    for i in range(iters):
        if ants.grad is not None:
            ants.requires_grad_()
            ants.grad.zero_()
        output = model(ants)
        output.backward()

        with torch.no_grad():
            last = ants.clone()
            noise = torch.rand_like(ants) * noise_std
            ants -= (lr * ants.grad * (ants != 0).float())
            if calculate_min_distance(ants) < 0.5:
                continue
        ants.grad.zero_()
    return ants


In [22]:
import matplotlib.pyplot as plt
def show_arrays(inp):
    data = inp.cpu().detach().numpy()
    reshaped_data = np.reshape(data, (2,1024))
    x = reshaped_data[0]
    y = reshaped_data[1]

    nonzero_ind = np.nonzero((x != 0) | (y != 0))
    x = x[nonzero_ind]
    y = y[nonzero_ind]

    plt.scatter(x,y, s=0.75)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Antenna Array Pattern')
    plt.show()

In [23]:
given_shapes =  np.zeros((20, 2048))#pd.DataFrame(columns=[f'y{i + 1}' for i in range(1024)] + [f'z{i + 1}' for i in range(1024)])
optimized_shapes = np.zeros((20, 2048)) #pd.DataFrame(columns=[f'y{i + 1}' for i in range(1024)] + [f'z{i + 1}' for i in range(1024)])
given_shapes_True_cost = []
given_shapes_pred_cost = []
optimized_shapes_pred_cost = []

In [ ]:
train_loader = DataLoader(AntDataset('data/large/scaled_cost/YZ_Large_70_Cost_Scaled.npz'))
i = 0
for inp, out in train_loader:
    if i == 20:
        break

    given_shapes_True_cost.append(out.item())
    given_shapes[i, :] = inp.flatten().numpy()
    inp, out = inp.to(device), out.to(device)

    inp_copy = inp.clone()
    inp_copy.requires_grad_()
    print(f'True Cost: {out}')
    show_arrays(inp)
    inp.requires_grad_()
    pred = model(inp)
    given_shapes_pred_cost.append(pred.item())
    print(f'Pred Cost: {pred}')
    print(calculate_min_distance(inp))
    optimized_inp = gradient_wrt_input(model, inp_copy,1e-2,1e-12, 10000)
    optimized_shapes[i, :] = optimized_inp.detach().flatten().cpu().numpy()
    optimized_inp.to(device)
    new_pred = model(optimized_inp)
    optimized_shapes_pred_cost.append(new_pred.item())
    print(f'New Pred: {new_pred}')
    show_arrays(optimized_inp)
    print(optimized_inp.shape)
    print(calculate_min_distance(optimized_inp))
    i += 1

In [233]:
columns = [f'y{i + 1}' for i in range(1024)] + [f'z{i + 1}' for i in range(1024)]
given_shapes_df = pd.DataFrame(given_shapes, columns=columns)
optimized_shapes_df = pd.DataFrame(optimized_shapes, columns=columns)

In [235]:
given_shapes_df.to_csv('given_shapes.csv', index = False)
optimized_shapes_df.to_csv('optimized_shapes.csv', index = False)

In [237]:
given_shapes_true_cost_df = pd.DataFrame({'True Cost': given_shapes_True_cost})
given_shapes_pred_cost_df = pd.DataFrame({'Predicted Cost': given_shapes_pred_cost})
optimized_shapes_pred_cost_df = pd.DataFrame({'Optimized Predicted Cost': optimized_shapes_pred_cost})

In [239]:
given_shapes_pred_cost_df.to_csv('given_shapes_pred_cost.csv', index = False)
given_shapes_true_cost_df.to_csv('given_shapes_true_cost.csv', index = False)
optimized_shapes_pred_cost_df.to_csv('optimized_shapes_pred_cost.csv', index = False)

In [4]:
data = np.load('data/large/large_YZant.npz')['arr_0']

In [2]:
small, large = np.load('data/large/scaled_cost/YZ_Large_30_Cost_Scaled.npz')['data'], np.load('data/large/scaled_cost/YZ_Large_70_Cost_Scaled.npz')['data']


In [6]:
stacked = np.vstack((small, large))

In [9]:
indices = np.argsort(stacked[:, -1])
best_10 = stacked[indices[:10]]

In [10]:
best_10

array([[-30.64484559, -29.70205211, -28.75925864, ...,   0.        ,
          0.        ,  -3.21482132],
       [-30.75978435, -29.21376344, -27.66774254, ...,   0.        ,
          0.        ,  -3.15755776],
       [-31.07106781, -29.65685425, -28.24264069, ...,   0.        ,
          0.        ,  -3.13994073],
       ...,
       [-30.38340047, -28.72046124, -27.05752202, ...,   0.        ,
          0.        ,  -3.02542015],
       [-30.64484559, -29.70205211, -28.75925864, ...,   0.        ,
          0.        ,  -3.02417421],
       [-30.38340047, -28.72046124, -27.05752202, ...,   0.        ,
          0.        ,  -3.01451864]])

In [12]:
columns = [f'y{i + 1}' for i in range(1024)] + [f'z{i + 1}' for i in range(1024)] + ['True Cost']
top_10_arrays = pd.DataFrame(best_10, columns=columns)
top_10_arrays

,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,...,z1016,z1017,z1018,z1019,z1020,z1021,z1022,z1023,z1024,True Cost
0,-30.644846,-29.702052,-28.759259,-30.766590,-29.823797,-28.881003,-27.938210,-26.995416,-26.052623,-29.945541,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.214821
1,-30.759784,-29.213763,-27.667743,-31.037019,-29.490998,-27.944977,-26.398956,-24.852935,-23.306914,-29.768232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.157558
2,-31.071068,-29.656854,-28.242641,-26.828427,-31.071068,-29.656854,-28.242641,-26.828427,-25.414214,-24.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.139941
3,-30.759784,-29.213763,-27.667743,-31.037019,-29.490998,-27.944977,-26.398956,-24.852935,-23.306914,-29.768232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.108225
4,-30.759784,-29.213763,-27.667743,-31.037019,-29.490998,-27.944977,-26.398956,-24.852935,-23.306914,-29.768232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.103843
5,-29.945541,-28.181698,-26.417856,-30.766590,-29.002747,-27.238905,-25.475062,-23.711220,-21.947377,-29.823797,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.098636
6,-30.383400,-28.720461,-27.057522,-30.935199,-29.272260,-27.609321,-25.946382,-24.283442,-22.620503,-29.824059,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.061418
7,-30.383400,-28.720461,-27.057522,-30.935199,-29.272260,-27.609321,-25.946382,-24.283442,-22.620503,-29.824059,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.025420
8,-30.644846,-29.702052,-28.759259,-30.766590,-29.823797,-28.881003,-27.938210,-26.995416,-26.052623,-29.945541,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.024174
9,-30.383400,-28.720461,-27.057522,-30.935199,-29.272260,-27.609321,-25.946382,-24.283442,-22.620503,-29.824059,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.014519


In [14]:
top_10_arrays.to_csv('top_10_arrays.csv', index=False)

In [40]:
# Load model
model = NN2(2048, 1, 8, 8, nn.ReLU)
model.load_state_dict(torch.load('weights/NN2_CostScaled_8_8_ReLU.pth'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

NN2(
  (regressor): Sequential(
    (0): Linear(in_features=2048, out_features=8, bias=True)
    (1): ReLU()
    (2): Dropout(p=0, inplace=False)
    (3): Linear(in_features=8, out_features=8, bias=True)
    (4): ReLU()
    (5): Dropout(p=0, inplace=False)
    (6): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [41]:
given_shapes =  np.zeros((10, 2048))#pd.DataFrame(columns=[f'y{i + 1}' for i in range(1024)] + [f'z{i + 1}' for i in range(1024)])
optimized_shapes = np.zeros((10, 2048)) #pd.DataFrame(columns=[f'y{i + 1}' for i in range(1024)] + [f'z{i + 1}' for i in range(1024)])
given_shapes_True_cost = []
given_shapes_pred_cost = []
optimized_shapes_pred_cost = []

In [42]:

train_loader = DataLoader(AntDataset(best_10))
i = 0
for inp, out in train_loader:

    given_shapes_True_cost.append(out.item())
    given_shapes[i, :] = inp.flatten().numpy()
    inp, out = inp.to(device), out.to(device)

    inp_copy = inp.clone()
    inp_copy.requires_grad_()
    print(f'True Cost: {out}')
    inp.requires_grad_()
    pred = model(inp)
    given_shapes_pred_cost.append(pred.item())
    print(f'Pred Cost: {pred}')
    optimized_inp = gradient_wrt_input(model, inp_copy,1e-2,1e-12, 10000)
    optimized_shapes[i, :] = optimized_inp.detach().flatten().cpu().numpy()
    optimized_inp.to(device)
    new_pred = model(optimized_inp)
    optimized_shapes_pred_cost.append(new_pred.item())
    print(f'New Pred: {new_pred}')
    i += 1

True Cost: tensor([-3.2148], device='cuda:0')
Pred Cost: tensor([[-2.7861]], device='cuda:0', grad_fn=<AddmmBackward0>)
New Pred: tensor([[-4.5998]], device='cuda:0', grad_fn=<AddmmBackward0>)
True Cost: tensor([-3.1576], device='cuda:0')
Pred Cost: tensor([[-2.6941]], device='cuda:0', grad_fn=<AddmmBackward0>)
New Pred: tensor([[-3.9437]], device='cuda:0', grad_fn=<AddmmBackward0>)
True Cost: tensor([-3.1399], device='cuda:0')
Pred Cost: tensor([[-2.7631]], device='cuda:0', grad_fn=<AddmmBackward0>)
New Pred: tensor([[-3.7884]], device='cuda:0', grad_fn=<AddmmBackward0>)
True Cost: tensor([-3.1082], device='cuda:0')
Pred Cost: tensor([[-2.8222]], device='cuda:0', grad_fn=<AddmmBackward0>)
New Pred: tensor([[-3.9638]], device='cuda:0', grad_fn=<AddmmBackward0>)
True Cost: tensor([-3.1038], device='cuda:0')
Pred Cost: tensor([[-2.8985]], device='cuda:0', grad_fn=<AddmmBackward0>)
New Pred: tensor([[-4.1084]], device='cuda:0', grad_fn=<AddmmBackward0>)
True Cost: tensor([-3.0986], device

In [43]:
from antenna_array_conversion.true_function import *

In [45]:
def reformat_ant_format(ant):
    """
    Reformats the 2048 to 2 distinct arrays, trimming the 0
    """
    Yant, Zant = ant[:1024], ant[1024:]
    trim_yant, trim_zant = np.trim_zeros(Yant, 'b'), np.trim_zeros(Zant, 'b')
    return trim_yant, trim_zant

In [53]:
given_shapes_true_cost = []
optimized_shapes_true_cost = []
ParamsArray = InitializeParamsArray()

for i in range(10):
    given_s = given_shapes[i]
    optim_s = optimized_shapes[i]

    # reformat the two
    Yant, Zant = reformat_ant_format(given_s)
    Op_Yant, Op_Zant = reformat_ant_format(optim_s)
    give_cost = CostFunArray(Yant, Zant, ParamsArray)
    opt_cost = CostFunArray(Op_Yant, Op_Zant, ParamsArray)
    given_shapes_true_cost.append(give_cost)
    optimized_shapes_true_cost.append(opt_cost)

In [54]:
given_shapes_true_cost

[-40463.982498152494,
 -39931.30801885433,
 -39767.25195126816,
 -39472.3885803695,
 -39431.63349183295,
 -39383.21482018078,
 -39036.94416721868,
 -38702.14763550828,
 -38690.556632721054,
 -38600.71619820203]

In [55]:
optimized_shapes_true_cost

[-58710.18204780458,
 -59040.24047398961,
 -54962.24929441841,
 -56971.855385250004,
 -59919.723320660254,
 -57293.98497793798,
 -56158.55941964273,
 -56021.402214458685,
 -57170.88897982669,
 -53393.262988085364]

In [56]:
with open('weights/scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [64]:
scaled_given_pred_cost = scaler.inverse_transform([given_shapes_pred_cost]).flatten()
scaled_opt_pred_cost = scaler.inverse_transform([optimized_shapes_pred_cost]).flatten()

In [67]:
compiled_shape_costs = pd.DataFrame(columns=['Given Shape - True Cost', 'Given Shape - Predicted Cost', 'Optimized Shape - True Cost', 'Optimized Shape - Predicted Cost'])
compiled_shape_costs['Given Shape - True Cost'] = given_shapes_true_cost
compiled_shape_costs['Given Shape - Predicted Cost'] = scaled_given_pred_cost
compiled_shape_costs['Optimized Shape - True Cost'] = optimized_shapes_true_cost
compiled_shape_costs['Optimized Shape - Predicted Cost'] = scaled_opt_pred_cost

In [68]:
compiled_shape_costs

,Given Shape - True Cost,Given Shape - Predicted Cost,Optimized Shape - True Cost,Optimized Shape - Predicted Cost
0,-40463.982498,-36476.521966,-58710.182048,-53347.106459
1,-39931.308019,-35620.608276,-59040.240474,-47243.831831
2,-39767.251951,-36262.105538,-54962.249294,-45799.087986
3,-39472.388580,-36812.156130,-56971.855385,-47430.885670
4,-39431.633492,-37521.494341,-59919.723321,-48775.881809
5,-39383.214820,-36278.383817,-57293.984978,-46749.903600
6,-39036.944167,-36404.201165,-56158.559420,-46670.996084
7,-38702.147636,-33512.180805,-56021.402214,-48064.117381
8,-38690.556633,-35911.104590,-57170.888980,-45365.034548
9,-38600.716198,-35203.695822,-53393.262988,-44148.550322


In [71]:
new_ord = [0, 2, 1, 3]
compiled = compiled_shape_costs.iloc[:, new_ord]
compiled.to_csv('Best 10 Shapes - Optimized.csv', index= False)